In [ ]:
# Single timestep evaluation of RK3 method.

import numpy as np


def step_RK3(evalf, dt, vn, tn):
    """
    Takes a single timestep of the 3rd order Runge-Kutta method (RK3) to
    integrate the state from vn (at time tn) to vn1 (at time tn1 = tn + dt).

    Args:
        evalf (function reference): the referenced function has inputs of
            a state vector (v) and time (t) and returns the forcing f(v, t).
            v and f(v, t) are NumPy arrays of floats.
        dt (float): time increment
        vn (NumPy array of floats): current state
        tn (float): current time

    Returns:
        vn1 (NumPy array of floats): next state
    """
    #### BEGIN SOLUTION ####
    a = dt * evalf(vn, tn)  # float array
    b = dt * evalf(vn + a, tn + dt)  # float array
    c = dt * evalf(vn + 0.25*a + 0.25*b, tn + 0.5*dt)  # float array
    vn1 = vn + (a + b + 4*c)/6  # float array
    
    return vn1
    
    #### END SOLUTION ####

In [ ]:
import numpy as np


"""
The docstring for both calc_methodA and calc_methodB follows:

This function calculates the residual r and the Jacobian matrix r_v
for an implicit numerical method.

Args:
    vnp1 (NumPy array of floats): guess for state at time t^{n+1} = tn + dt
    vn (NumPy array of floats): state at time tn
    vnm1 (NumPy array of floats): state at time t^{n-1} = tn - dt

    dt (float): time increment
    tn (float): time at step n

    evalf (function reference): the referenced function has inputs of
        a state vector (v) and time (t) and returns the forcing f(v, t).
        v and f(v, t) are NumPy arrays of floats.

    evalf_v (function reference): the referenced function has inputs of
        a state vector (v) and time (t) and returns the matrix of
        derivatives, df/dv, where f is the forcing f(v, t).
        The matrix of derivatives returned by evalf_v are in a 2D NumPy
        array of floats such that the following line:

            f_v = evalf_v(v, t)

        will result in f_v being a 2D NumPy array where f_v[i,j] is the
        derivative of the i'th component of f with respect to the j'th
        state, i.e. d(f_i)/d(v_j).

Returns: tuple (r, r_v) where
    r (NumPy array of floats): residual vector
    r_v (2D NumPy array of floats): Jacobian matrix where r_v[i,j] is
        the derivative of the i'th component of r with respect to the j'th
        state, i.e. d(r_i)/d(v_j)
"""


def calc_methodA(vnp1, vn, vnm1, dt, tn, evalf, evalf_v):
    """
    Implementation for numerical method A.

    See docstring given at the top of this file.
    """
    #### BEGIN SOLUTION ####
    r = (1/dt)*(vnp1 + (-4/3)*vn + (1/3)*vnm1) - (2/3)*evalf(vnp1, tn + dt)
    
    I = np.eye(len(vn))
    J = evalf_v(vnp1, tn + dt)
    r_v = (1/dt)*I - (2/3)*J
    
    return (r, r_v)
    #### END SOLUTION ####


def calc_methodB(vnp1, vn, vnm1, dt, tn, evalf, evalf_v):
    """
    Implementation for numerical method B.

    See docstring given at the top of this file.
    """
    #### BEGIN SOLUTION ####
    r = (1/dt)*(vnp1 - vn) - ((5/12)*evalf(vnp1, tn + dt) + (8/12)*evalf(vn, tn) - (1/12)*evalf(vnm1, tn - dt))
    
    I = np.eye(len(vn))
    J = evalf_v(vnp1, tn + dt)
    r_v = (1/dt)*I - (5/12)*J
    
    return (r, r_v)
    #### END SOLUTION ####

In [1]:
################################################################################
# CSE.0002x
# Exam 1: IVPlib
# DO NOT MODIFY THIS FILE

"""
This Python library is useful in solving Initial Value Problems (IVP).

It implements an IVP base class that defines an IVP in the form:

    du/dt = f(u, t, p)   for u(tI) = uI,

in which the problem would be solved from t=tI to tF, and p is a set of
parameters.
"""

import copy
import numpy as np
from scipy import optimize


class IVP():

    def __init__(self, uI, tI, tF, p={}):
        """
        Args:
            uI (NumPy ndarray): initial condition of state
            tI (float): initial time
            tF (float): final time
            p (dictionary): set of fixed parameters
        """
        self.__uI = uI.copy()
        self.__tI = tI
        self.__tF = tF
        self.__p = copy.deepcopy(p)

    ############ (other) dunder methods ############

    def __len__(self):
        """
        len is defined as number of states (in _uI)
        """
        return len(self.__uI)

    ############ virtual methods for use outside of class ############

    def evalf(self, u, t):
        """
        Calculates the right-hand side f(u, t, p) for the IVP

        Args:
            u (NumPy ndarray): current solution
            t (float): current time

        NOTE: Any needed parameter values (i.e. p) should be found by calling
        the get_p method defined below.

        Returns:
            f (NumPy ndarray): the evaluation of f(u, t, p)
        """
        raise NotImplementedError("evalf is not implemented for this class")

    def evalf_u(self, u, t):
        """
        Calculates the Jacobian df/du of the right-hand side f(u, t, p)

        Args:
            u (NumPy ndarray): current state
            t (float): current time

        NOTE: Any needed parameter values (i.e. p) should be found by calling
        the get_p method defined below.

        Returns:
            f_u (NumPy 2D ndarray): Jacobian of current right-hand side where
                f_u[i,j] = the derivative of f[i] with respect to u[j]
        """
        raise NotImplementedError("evalf_u is not implemented for this class")

    ############ getter methods ############

    def get_tI(self):
        """
        Returns:
            float: initial time
        """
        return self.__tI

    def get_tF(self):
        """
        Returns:
            float: final time
        """
        return self.__tF

    def get_uI(self):
        """
        Returns:
            NumPy ndarray: initial state
        """
        return self.__uI.copy()

    def get_p(self, name):
        """
        Arg:
            name (key): a key which should be in the object's parameter
                dictionary

        Returns:
            value of parameter key given by name
        """
        return self.__p[name]


################################################################################
## Functions to numerically integrate an IVP
################################################################################


def step_FE(thisIVP, dt, un, tn):
    """
    Takes a single timestep of the Forward Euler method (FE) to
    (approximately) integrate the state from u(tn) to u(tn+dt)

    Args:
        thisIVP (IVP object): object describing IVP being solved
        dt (float): time increment
        un (NumPy ndarray): current state, i.e. u(tn)
        tn (float): current time

    Returns:
        NumPy ndarray: next state, i.e. u(tn+dt)
    """
    fn = thisIVP.evalf(un, tn)
    un1 = un + dt*fn
    return un1


def step_RK2_ME(thisIVP, dt, un, tn):
    """
    Takes a single timestep of the Modified Euler version of a 2nd order
    Runge-Kutta method (RK2_ME) to (approximately) integrate the state from
    u(tn) to u(tn+dt)

    Args:
        thisIVP (IVP object): object describing IVP being solved
        dt (float): time increment
        un (NumPy ndarray): current state, i.e. u(tn)
        tn (float): current time

    Returns:
        NumPy ndarray: next state, i.e. u(tn+dt)
    """
    # compute the a vector
    a = dt*thisIVP.evalf(un, tn)

    # compute the b vector
    b = dt*thisIVP.evalf(un + 0.5*a, tn + 0.5*dt)

    # compute the next step
    un1 = un + b
    return un1


def step_BE(thisIVP, dt, un, tn):
    """
    Takes a single timestep of the Backward Euler method (BE) to
    (approximately) integrate the state from u(tn) to u(tn+dt)

    Args:
        thisIVP (IVP object): object describing IVP being solved
        dt (float): time increment
        un (NumPy ndarray): current state, i.e. u(tn)
        tn (float): current time

    Returns:
        NumPy ndarray: next state, i.e. u(tn+dt)
    """
    def evalr(v):
        return (v - un)/dt - thisIVP.evalf(v, tn+dt)

    def evalr_u(v):
        return np.eye(len(v))/dt - thisIVP.evalf_u(v, tn+dt)

    # Use Newton's method to determine un1
    un1root = optimize.root(evalr, un, jac=evalr_u)
    un1 = un1root.x
    return un1


def solve(thisIVP, dt, method):
    """
    Solves an IVP using a timestep dt and method. Integrate from t=tI until u(tn) is
    determined for which tn >= tF.

    Args:
        thisIVP (IVP object): object describing IVP being solved
        dt (float): time increment
        method (function): numerical integration method to use

    Returns:
        t (NumPy ndarray): time values at which u(t) is approximated. The nth item in
            the list is the time of the nth step, tn = t[n].
        u (NumPy ndarray): the values of the states at each step. u(tn) = u[n].
            So, if there are three equations being integrated, then u[n,0],
            u[n,1], and u[n,2] are the values of the three states at time t=t[n].

    IMPORTANT: The first element in the returned t and u lists will be the initial values
    of t and u. Thus:
        t[0] will be a float which is equal to thisIVP.get_tI()
        u[0] will be an NumPy ndarray which is equal to thusIVP.get_uI()
    """
    # Set initial conditions
    tI = thisIVP.get_tI()
    tF = thisIVP.get_tF()
    uI = thisIVP.get_uI()

    M = len(uI) # number of states
    t = np.arange(tI, tF+dt, dt)
    N = len(t) # number of time points
    u = np.zeros((N, M))
    u[0, :] = uI

    # Loop from t=tI to t>=tF
    for n in range(1, N):
        u[n, :] = method(thisIVP, dt, u[n-1, :], t[n-1])

    return t, u


In [ ]:
################################################################################
# CSE.0002x
# Exam 1: osccomb2
# edX username: adevrent

import numpy as np
from IVPlib import IVP


################################################################################
## Oscillating combustor fuel+oxidizer class definition (a subclass of IVP)
################################################################################


class OscComb2IVP(IVP):
    """
    The OscComb2IVP class is derived from the IVP base class. The
    states in this IVP are:
        u[0] = fuel concentration
        u[1] = oxidizer concentration

    The following keys must be used for the parameters in this IVP:
        'tau':      combustion timescale
        'A_fuel':   fuel oscillation amplitude
        'T_fuel':   fuel oscillation time period
        'C_ox':     oxidizer reaction rate coefficient
        'ox_base':  oxidizer base concentration
    The values for these parameters should be retrieved using the get_p
    method defined in the IVP base class.
    """

    #### BEGIN SOLUTION #####
    def evalf(self, un, tn):
        """Evaluates the forcing function at current state and time for combustion class.

        Args:
            un (ndarray): current state
            tn (float): current time

        Returns:
            f (ndarray): value of forcing function at current state and time
        """
        tau = self.get_p("tau")
        A_fuel = self.get_p("A_fuel")
        T_fuel = self.get_p("T_fuel")
        C_ox = self.get_p("C_ox")
        ox_base = self.get_p("ox_base")
        
        f = np.zeros(2)
        f[0] = -(1/tau)*un[0]*un[1] + (1/2)*A_fuel*(1 - np.cos(2*np.pi*(tn/T_fuel))) # might need to convert to radians
        f[1] = -C_ox*(1/tau)*un[0]*un[1] + ox_base - un[1]
        
        return f
    #### END SOLUTION #####

    #### BEGIN SOLUTION #####
    def evalf_u(self, un, tn):
        """Calculates the Jacobian df/du

        Args:
            un (ndarray): current state
            tn (float): current time
        
        Returns:
            J (square ndarray): Jacobian matrix of the function f. (df/du)
        """
        tau = self.get_p("tau")
        A_fuel = self.get_p("A_fuel")
        T_fuel = self.get_p("T_fuel")
        C_ox = self.get_p("C_ox")
        ox_base = self.get_p("ox_base")
        
        J = np.eye(len(un))
        J[0, 0] = (-1/tau) * un[1]
        J[1, 0] = -C_ox * (1/tau) * un[1]
        J[0, 1] = -(1/tau) * un[0]
        J[1, 1] = -C_ox * (1/tau) * un[0] - 1
        
        return J
        
    #### END SOLUTION #####
